In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

c:\Users\haokw\.conda\envs\cat_dog\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [5]:

X_train = []
X_test = []

for filename in ["D:\data\kaggle\cat_dog\my_gap\\gap_my_ResNet50.h5", 
                    "D:\data\kaggle\cat_dog\my_gap\\gap_my_Xception.h5", 
                    "D:\data\kaggle\cat_dog\my_gap\\gap_my_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [6]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# 使用load_model函数加载模型
model = load_model('result\\model_my_best_3.h5')
# 查看模型的架构
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6144)]            0         
                                                                 
 dropout (Dropout)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 1)                 6145      
                                                                 
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train, y_train, batch_size=512, epochs=10, validation_split=0.2)

Epoch 1/10
40/40 [==============================] - 2s 33ms/step - loss: 0.0196 - accuracy: 0.9935 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 2/10
40/40 [==============================] - 1s 22ms/step - loss: 0.0200 - accuracy: 0.9930 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 3/10
40/40 [==============================] - 1s 22ms/step - loss: 0.0204 - accuracy: 0.9940 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 4/10
40/40 [==============================] - 1s 22ms/step - loss: 0.0195 - accuracy: 0.9933 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 5/10
40/40 [==============================] - 1s 21ms/step - loss: 0.0209 - accuracy: 0.9930 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 6/10
40/40 [==============================] - 1s 22ms/step - loss: 0.0201 - accuracy: 0.9937 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 7/10
40/40 [==============================] - 1s 23ms/step - loss: 0.0189 - accuracy: 0.9940 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 8/10
40

In [ ]:
model.save('result\\model_my_5.h5')

In [10]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

391/391 [==============================] - 0s 762us/step


In [11]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("result\\pred.csv")
df2 = pd.read_csv("result\\pred_name.csv")
image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("D:\data\cat_dog\test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.at[index-1, 'label'] = y_pred[i]
    if y_pred[i]>0.50:
        df2.at[index-1, 'label'] = "狗"
    else:
        df2.at[index-1, 'label'] = "猫"


df.to_csv('result\\pred.csv', index=None)
df.head(10)
df2.to_csv('result\\pred_name.csv', index=None)
df2.head(10)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:\\data\\cat_dog\\test'